In [1]:
%load_ext autoreload
%autoreload 2
from utils.predict_np import predict_from_np
from shapley.shapley_fda import ShapleyFda
from skfda.ml.regression import LinearRegression
from skfda.representation.basis import MonomialBasis
from skfda.representation.grid import FDataGrid
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Prepare the data
np.random.seed(1234)
num_functions = 100
rate_important_intervals = 0.7
n_basis = 10
basis = MonomialBasis(n_basis=n_basis)
ini = 0
end = 1
domain_range = (ini, end)
step = 0.05
abscissa_points = np.arange(ini, end + step, step)
total_abscissa_points = abscissa_points.shape[0]
total_important_abscissa = int(np.floor(rate_important_intervals * abscissa_points.shape[0]))
col_vector_ones_num_functions = np.full(shape=(num_functions, 1), fill_value=1)
row_vector_ones_total_important_abscissa = np.full(shape=(1, total_important_abscissa), fill_value=1)
vector_fun = np.array([x ** 3 for x in abscissa_points[:total_important_abscissa]], ndmin=2)
X_no_noise = np.dot(col_vector_ones_num_functions, vector_fun) * (1 + np.random.normal(loc=0, scale=0.2, size=(num_functions, total_important_abscissa)))
X_last_abs = np.dot(np.reshape(X_no_noise[:, -1], newshape=(-1, 1)), row_vector_ones_total_important_abscissa)
X_no_noise = np.divide(X_no_noise, X_last_abs)
#X_no_noise = np.dot(vector_col_ones_num_functions, vector_fun) + np.random.normal(loc=0, scale=0.001, size=(num_functions, total_important_abscissa))
X_last_abscissa = np.reshape(X_no_noise[:, -1], newshape=(-1, 1))
X_noise = np.random.normal(loc=X_last_abscissa, scale= 0.01, size=(num_functions, total_abscissa_points - total_important_abscissa))
y = np.empty(shape=num_functions)
for i in range(num_functions):
    y[i] = np.dot(X_no_noise[i, :], X_no_noise[i, :])
y = y - np.mean(y)
X = np.column_stack((X_no_noise, X_noise))
X_grid = FDataGrid(data_matrix=X, grid_points=abscissa_points, domain_range=domain_range)
X_basis = X_grid.to_basis(basis)
# Derivative
derivative_X = np.squeeze(X_basis.derivative().to_grid().data_matrix)
# Plot
zzz = X_grid.plot()
yyy = X_basis.plot()

In [ ]:
# Fit the model
linear_reg = LinearRegression()
_ = linear_reg.fit(X_basis, y)

In [ ]:
predict_function = predict_from_np(
    grid_points=abscissa_points,
    domain_range=domain_range,
    basis=basis,
    predict_fn = linear_reg.predict
)

In [ ]:
y_pred = predict_function(X)

In [ ]:
plt.scatter(y_pred, y)

In [ ]:
shapley_fda = ShapleyFda(
    predict_fn=predict_function,
    X=X,
    derivative_X=derivative_X,
    abscissa_points=abscissa_points,
    target=y,
    domain_range=domain_range,
    verbose=True,
)
values_shapley = shapley_fda.compute_shapley_value(num_intervals=5, num_permutations=1)
#print("relevance:\n", values_shapley, "\n")

In [ ]:
print("relevance:\n", values_shapley, "\n")

In [ ]:
np.power(0.1, 1/10)

In [ ]:
np.power(2,3)